In [52]:
import pandas as pd
from sqlalchemy import create_engine

In [53]:
episodes = pd.read_csv("Resources/datasets_631837_1124849_GOT_episodes_v4.csv")
deaths = pd.read_csv("Resources/game-of-thones-deaths.csv")

In [70]:
episodes_cols = ["Season", "Episode", "Title", "Rating", "Release_date"]
episodes_t = episodes[episodes_cols].copy()
episodes_t

,Season,Episode,Title,Rating,Release_date
0,1,1,Winter Is Coming,9.1,17-Apr-11
1,1,2,The Kingsroad,8.8,24-Apr-11
2,1,3,Lord Snow,8.7,1-May-11
3,1,4,"Cripples, Bastards, and Broken Things",8.8,8-May-11
4,1,5,The Wolf and the Lion,9.1,15-May-11
...,...,...,...,...,...
68,8,2,A Knight of the Seven Kingdoms,7.9,21-Apr-19
69,8,3,The Long Night,7.5,28-Apr-19
70,8,4,The Last of the Starks,5.5,5-May-19
71,8,5,The Bells,6.0,12-May-19


In [55]:
deaths_cols = ["Season", "Episode", "Killer"]
deaths_t = deaths[deaths_cols].copy()
deaths_t

,Season,Episode,Killer
0,1,1,White Walker
1,1,1,White Walker
2,1,1,Ned Stark
3,1,1,Direwolf
4,1,1,Stag
...,...,...,...
2219,8,5,Daenerys Targaryen
2220,8,5,Daenerys Targaryen
2221,8,5,Daenerys Targaryen
2222,8,5,Daenerys Targaryen


In [71]:
episodes_t["episode_id"] = episodes_t["Season"].map(str) + '-' + episodes_t["Episode"].map(str)




episodes_t.head()

,Season,Episode,Title,Rating,Release_date,episode_id
0,1,1,Winter Is Coming,9.1,17-Apr-11,1-1
1,1,2,The Kingsroad,8.8,24-Apr-11,1-2
2,1,3,Lord Snow,8.7,1-May-11,1-3
3,1,4,"Cripples, Bastards, and Broken Things",8.8,8-May-11,1-4
4,1,5,The Wolf and the Lion,9.1,15-May-11,1-5


In [80]:
episodes_t2 = episodes_t.rename(columns = {"Season" : 'season', "Episode" : 'episode', "Title" : 'title', "Rating" : 'rating', 
                                          "Release_date" : "release_date"})
episodes_t3 = episodes_t2[["episode_id", "rating", "title", "release_date", "season", "episode"]]
episodes_t3

,episode_id,rating,title,release_date,season,episode
0,1-1,9.1,Winter Is Coming,17-Apr-11,1,1
1,1-2,8.8,The Kingsroad,24-Apr-11,1,2
2,1-3,8.7,Lord Snow,1-May-11,1,3
3,1-4,8.8,"Cripples, Bastards, and Broken Things",8-May-11,1,4
4,1-5,9.1,The Wolf and the Lion,15-May-11,1,5
...,...,...,...,...,...,...
68,8-2,7.9,A Knight of the Seven Kingdoms,21-Apr-19,8,2
69,8-3,7.5,The Long Night,28-Apr-19,8,3
70,8-4,5.5,The Last of the Starks,5-May-19,8,4
71,8-5,6.0,The Bells,12-May-19,8,5


In [86]:
deaths_t["episode_id"] = deaths_t["Season"].map(str) + '-' + deaths_t["Episode"].map(str)
deaths_t.head()

,Season,Episode,Killer,episode_id
0,1,1,White Walker,1-1
1,1,1,White Walker,1-1
2,1,1,Ned Stark,1-1
3,1,1,Direwolf,1-1
4,1,1,Stag,1-1


In [89]:
deaths_gb = deaths_t.groupby('episode_id')
deaths_gb

In [90]:
deaths_sum = deaths_gb["Killer"].count()
deaths_t2 = pd.DataFrame({'death_count' : deaths_sum})
deaths_t2

,death_count
episode_id,
1-1,7
1-10,3
1-2,3
1-4,1
1-5,17
...,...
8-2,1
8-3,35
8-4,5


In [45]:
connection_string = "postgres:test123@localhost:5432/got_db"
engine = create_engine(f'postgresql://{connection_string}')

In [46]:
engine.table_names()

['episodes', 'deaths']

In [85]:
episodes_t3.to_sql(name='episodes', con=engine, if_exists='append', index=False)

In [92]:
deaths_t2.to_sql(name='deaths', con=engine, if_exists='append', index=True)

In [93]:
deaths_t2

,death_count
episode_id,
1-1,7
1-10,3
1-2,3
1-4,1
1-5,17
...,...
8-2,1
8-3,35
8-4,5
